# Accelerating End-to-End Data Science Workflows # 

## 07 - Triton ##

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้จะอธิบายถึงกระบวนการการปรับใช้โมเดลด้วย Triton Inference Server และพาคุณไปทำความเข้าใจเครื่องมือต่างๆ ที่มีอยู่ใน Triton โดยครอบคลุมหัวข้อด้านล่างนี้:
1. [ความเป็นมา](#s1)
2. [การเตรียมโมเดล](#s1)
    * [โหลดโมเดล](#s1)
    * [การสร้างโครงสร้างโฟลเดอร์](#s1)
    * [การสร้างไฟล์กำหนดค่า](#s1)
3. [การโหลดโมเดลใน Triton](#s1)
    * [การเริ่มต้น Triton](#s1)
    * [ตรวจสอบสถานะของ Triton Server](#s1)
4. [การทดสอบการอนุมาน (Inference)](#s1)
    * [Triton Client](#s1)
    * [การตรวจสอบผลลัพธ์สำหรับโมเดลโลคัลและ Triton](#s1)
5. [การวิเคราะห์ประสิทธิภาพ](#s1)
    * [การปรับแต่ง Perf Analyzer](#s1)
    * [แบบฝึกหัดที่ 1 - การทดสอบ Perf Analyzer](#s1)
6. [Model Analyzer](#s1)

# ความเป็นมา
NVIDIA มีเฟรมเวิร์กสำหรับการนำโมเดล ML ไปใช้งานที่เรียกว่า **Triton** โดย Triton จะจัดการคำขอ Inference (การอนุมานผล) ทั้งหมดที่เข้ามายังเซิร์ฟเวอร์โดยอัตโนมัติ Triton รองรับแบ็กเอนด์หลายตัว เช่น PyTorch, TensorFlow, Forest Inference Library (FIL) เป็นต้น ในหน้านี้ (notebook) เราจะเน้นไปที่ **FIL backend** โดยใช้โมเดล xgboost ที่ได้ฝึกฝนไว้ก่อนหน้านี้

## การเตรียมโมเดล
### โหลดโมเดล
มาเริ่มต้นด้วยการโหลดโมเดล XGBoost ตัวก่อนหน้ากันครับ

In [ ]:
import xgboost as xgb
model = xgb.Booster({'nthread': 4})  # init model
model.load_model('xgboost_model.json')  # load model data

### สร้างโครงสร้างโฟลเดอร์
ใน Jupyter Notebook ก่อนหน้า เราได้บันทึกโมเดล XGBoost ไว้ใน working directory เท่านั้น แต่ Triton ต้องการให้โมเดลอยู่ในโครงสร้างเฉพาะ: **ชื่อโมเดล** ควรเป็นไดเรกทอรีระดับบนสุด และ **หมายเลขเวอร์ชัน** ควรอยู่ถัดไป ซึ่งช่วยให้สามารถโฮสต์โมเดลและเวอร์ชันต่างๆ ของโมเดลเหล่านั้นได้พร้อมกัน มาสร้างโครงสร้างโฟลเดอร์และบันทึกโมเดลกันเถอะ!

In [ ]:
import os

# Create the model repository directory. The name of this directory is arbitrary.
REPO_PATH = os.path.abspath('models')
os.makedirs(REPO_PATH, exist_ok=True)

# The name of the model directory determines the name of the model as reported by Triton
model_dir = os.path.join(REPO_PATH, "virus_prediction")

# We can store multiple versions of the model in the same directory. In our case, we have just one version, so we will add a single directory, named '1'.
version_dir = os.path.join(model_dir, '1')
os.makedirs(version_dir, exist_ok=True)

# The default filename for XGBoost models saved in json format is 'xgboost.json'.
# It is recommended that you use this filename to avoid having to specify a name in the configuration file.
model_file = os.path.join(version_dir, 'xgboost.json')
model.save_model(model_file)

### สร้างไฟล์การตั้งค่า (Configuration File)
Triton ยังต้องการ **ไฟล์การตั้งค่า** ที่ให้รายละเอียดเกี่ยวกับโมเดลและการนำไปใช้งาน สำหรับหน้านี้ (notebook) เราจะใช้ค่าพารามิเตอร์เริ่มต้น

In [ ]:
config_text = f"""backend: "fil"
max_batch_size: {32768}
input [                                 
 {{  
    name: "input__0"
    data_type: TYPE_FP32
    dims: [ 4 ]                    
  }} 
]
output [
 {{
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }}
]
instance_group [{{ kind: KIND_GPU }}]
parameters [
  {{
    key: "model_type"
    value: {{ string_value: "xgboost_json" }}
  }},
  {{
    key: "output_class"
    value: {{ string_value: "false" }}
  }},
  {{
    key: "storage_type"
    value: {{ string_value: "AUTO" }}
  }}
]

dynamic_batching {{
  max_queue_delay_microseconds: 100
}}"""
config_path = os.path.join(model_dir, 'config.pbtxt')
with open(config_path, 'w') as file_:
    file_.write(config_text)

ทีนี้โมเดลก็พร้อมที่จะโหลดลงใน Triton แล้ว! โครงสร้างของโมเดล (model repository) ควรมีลักษณะดังนี้ครับ
```
model/
`-- virus_prediction
    |-- 1
    |   `-- xgboost.model
    `-- config.pbtxt
```


## การโหลดโมเดลใน Triton
ถัดไป เราจะต้องเริ่ม **Triton server** สำหรับคอร์สเรียนนี้ เซิร์ฟเวอร์ได้ถูกเริ่มไว้แล้ว แต่เราจะกล่าวถึงขั้นตอนที่จำเป็นโดยย่อ

### การเริ่มต้นใช้งาน Triton

Triton มีให้เลือกใช้งานทั้งในรูปแบบซอร์สโค้ดที่สามารถคอมไพล์ได้ หรืออิมเมจ Docker ที่สร้างไว้ล่วงหน้าแล้ว เพื่อความง่าย เราแนะนำให้ผู้ใช้ส่วนใหญ่เริ่มต้นใช้งานด้วยอิมเมจ Docker นี่คือวิธีการที่คุณจะสามารถรัน Docker container ในคอนโซลได้

In [ ]:
#!docker run --gpus=1 --rm -p 8000:8000 -p 8001:8001 -p 8002:8002 -v /full/path/to/docs/examples/model_repository:/models nvcr.io/nvidia/tritonserver:<xx.yy>-py3 tritonserver --model-repository=/models

ว้าว! มี Inputs เยอะเลยใช่ไหมครับ มาทำความเข้าใจกันทีละส่วนดีกว่า:

* **gpus=1**: ส่งผ่าน GPU ตัวแรกไปยัง Triton Inference Server
* **rm**: ลบคอนเทนเนอร์หลังจากดำเนินการเสร็จสิ้น
* **p 8000:8000**: ส่งต่อพอร์ต GTPCInferenceService
* **p 8001:8001**: ส่งต่อพอร์ต HTTPService
* **p 8002:8002**: ส่งต่อพอร์ต Metrics
* **v /full/path/to/docs/examples/model_repository:/models**: เมาท์ (mount) พาธของโฟลเดอร์โมเดลบนเครื่องโฮสต์ไปยังคอนเทนเนอร์ Triton Inference Server
* **nvcr.io/nvidia/tritonserver:<xx.yy>-py3**: ชื่อของอิมเมจ Triton Inference Server หมายเลขเวอร์ชันจะเปลี่ยนไปตามรุ่นล่าสุดที่ออกมา
* **tritonserver --model-repository=/models**: คำสั่งที่ใช้รันในคอนเทนเนอร์ ในกรณีนี้ เราจะเริ่ม Triton Inference Server และชี้ไปยังโฟลเดอร์ `models`

ดังที่เราได้กล่าวไปก่อนหน้านี้ **เซิร์ฟเวอร์ได้ถูกเริ่มต้น** สำหรับแล็บนี้เรียบร้อยแล้ว **มาตรวจสอบการเชื่อมต่อ** ของเรากับเซิร์ฟเวอร์กัน! เราใช้ "triton" เป็นชื่อโฮสต์ (hostname) เนื่องจากเครือข่าย Docker เริ่มต้นจะทำการแปลง "triton" ให้เป็นที่อยู่ IP ของ Triton Inference Server ครับ

In [ ]:
!curl -v triton:8000/v2/health/ready

ตอนนี้มาดูกันว่าโมเดลโหลดเรียบร้อยหรือยัง!

In [ ]:
!curl -X POST http://triton:8000/v2/repository/index

หากทุกอย่างเป็นไปได้ด้วยดี เราก็น่าจะเห็นโมเดล **"การทำนายไวรัส (virus prediction)"** แสดงสถานะเป็น **พร้อมใช้งาน (ready)** ครับ

## การทดสอบการอนุมาน (Testing Inference)

### Triton Client
เพื่อทดสอบการติดตั้งใช้งาน (deployment) เราจะใช้ไลบรารี Triton Client มาดูกันว่าเราจะสร้างอินสแตนซ์ของไคลเอนต์ได้อย่างไร

In [ ]:
import time
import tritonclient.grpc as triton_grpc
from tritonclient import utils as triton_utils
HOST = "triton"
PORT = 8001
TIMEOUT = 60

In [ ]:
client = triton_grpc.InferenceServerClient(url=f'{HOST}:{PORT}')

ตอนนี้ เรามาตรวจสอบให้แน่ใจว่า Triton server พร้อมใช้งานแล้ว โดยการส่งคำขอ inference ตัวอย่าง ก่อนอื่นเรามาโหลดข้อมูลสำหรับ train (training data) เราจะโหลดเพียง **32768 แถว** เท่านั้น เนื่องจากเป็นขนาด batch สูงสุดที่เรากำหนดไว้

In [ ]:
import cudf 
import numpy as np
df = cudf.read_csv('./data/clean_uk_pop_full.csv', usecols=['age', 'sex', 'northing', 'easting', 'infected'], nrows=5000000)
df = df.sample(32768)
input_data = df.drop('infected', axis=1)
target = df[['infected']]
print(target)

ตอนนี้เราจะแปลงให้อยู่ในรูปแบบ **Numpy array** และกำหนดชนิดข้อมูลให้เป็น **float32** (ชนิดเดียวกันกับที่เรากำหนดไว้ในไฟล์ตั้งค่า)

In [ ]:
converted_df = input_data.to_numpy(dtype='float32')

เนื่องจากเราจำกัดขนาดแบตช์ (batch size) ไว้ที่ 32768 ดังนั้น เรามาทำการแบ่งอาร์เรย์ (splice the array) และลองทำการอนุมานผล (inference) กัน

In [ ]:
%%time
batched_data = converted_df[:32768]
# Prepare the input tensor
input_tensor = triton_grpc.InferInput("input__0", batched_data.shape, 'FP32')
input_tensor.set_data_from_numpy(batched_data)

# Prepare the output
output = triton_grpc.InferRequestedOutput("output__0")

# Send inference request
response = client.infer("virus_prediction", [input_tensor], outputs=[output])

# Get the output data
output_data = response.as_numpy("output__0")

มาดูกันว่าผลลัพธ์ที่เราได้เหมือนกับที่ใช้ **โมเดลโลคัล (local model)** หรือไม่

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

xgb_data = xgb.DMatrix(input_data)
y_test = model.predict(xgb_data)

In [ ]:
# Check that we got the same accuracy as previously
#target = target.to_numpy()
import matplotlib.pyplot as plt

false_pos_rate, true_pos_rate, thresholds = roc_curve(target.to_numpy(), y_test)
auc_result = auc(false_pos_rate, true_pos_rate)

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(false_pos_rate, true_pos_rate, lw=3,
        label='AUC = {:.2f}'.format(auc_result))
ax.plot([0, 1], [0, 1], 'k--', lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend(loc='lower right');
plt.show()


In [ ]:
# Check that we got the same accuracy as previously
#target = target.to_numpy()
import matplotlib.pyplot as plt

false_pos_rate, true_pos_rate, thresholds = roc_curve(target.to_numpy(), output_data)
auc_result = auc(false_pos_rate, true_pos_rate)

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(false_pos_rate, true_pos_rate, lw=3,
        label='AUC = {:.2f}'.format(auc_result))
ax.plot([0, 1], [0, 1], 'k--', lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend(loc='lower right');
plt.show()

อย่างที่เราเห็น **ค่า AUC** ของเราเท่ากันเลย ไม่ว่าจะใช้ตัวเลือกการอนุมาน (inference) แบบไหนก็ตาม!

## การวิเคราะห์ประสิทธิภาพ

ก่อนหน้านี้ เราได้ทดสอบการอนุมาน (inference) ด้วยคำขอที่ **ค่อนข้าง** เล็ก แล้วถ้าเราต้องการดู **อัตราการส่งข้อมูลสูงสุด (max throughput)** ของโมเดลล่ะ? โชคดีที่ Triton มีเครื่องมือวิเคราะห์ประสิทธิภาพที่สร้างข้อมูลจำลองขึ้นมาเพื่อเก็บตัวเลขความหน่วง (latency) และอัตราการส่งข้อมูล (throughput) งั้นเรามาลองใช้กันเลย!

In [ ]:
!perf_analyzer -m virus_prediction -u "triton:8000"

เป็นข้อมูลที่เยอะมากเลยทีเดียว ลองมาทำความเข้าใจไปทีละส่วนกัน

* **Measurement window**: **ช่วงเวลา** ที่ทำการวัดผล
* **Batch Size**: **จำนวนอินพุต** ในแต่ละคำขอ (request)
* **Concurrency**: **จำนวนการเชื่อมต่อพร้อมกัน**
* **Latency**: **เวลาที่ใช้ในการรับผลลัพธ์**
* **p50/90/95/99**: **เปอร์เซ็นไทล์ต่างๆ** สำหรับค่า Latency (เช่น p50 คือเปอร์เซ็นไทล์ที่ 50 หรือค่ามัธยฐาน)

จากผลลัพธ์เหล่านี้ เราจะเห็นได้ว่า **ปริมาณงาน (throughput)** ของเราอยู่ที่ประมาณ **~2300 การอนุมานต่อวินาที (inferences per second)** ด้วยการเชื่อมต่อพร้อมกันเพียงครั้งเดียว และ **ค่า Latency เฉลี่ย** สำหรับแต่ละคำขอคือ **434 ไมโครวินาที (usec)**

# การปรับแต่ง Perf Analyzer
เครื่องมือ Performance Analyzer สำหรับ Triton มีตัวเลือกมากมายที่สามารถปรับเปลี่ยนได้เพื่อวิเคราะห์ผลลัพธ์ มาเปิดใช้งานการเก็บ **เมตริก GPU** และเพิ่มค่า **ขนาดแบทช์ (batch size)** และ **ช่วงการทำงานพร้อมกัน (concurrency range)** กันเถอะ!

In [ ]:
!perf_analyzer --collect-metrics -m virus_prediction -u "triton:8000" -b 8 --concurrency-range 2:8:2

ผลลัพธ์แสดงให้เห็นว่าการตั้งค่าโมเดลของเราให้ปริมาณงาน (throughput) อยู่ที่ประมาณ ~156,171 การอนุมานต่อวินาที (inferences per second)

โปรดสังเกตว่ามีการเพิ่มขึ้นของปริมาณงานอย่างมีนัยสำคัญเมื่อเราเพิ่มจำนวนการเชื่อมต่อพร้อมกัน (concurrent connections)

ที่ค่า **concurrency** ต่ำ Triton จะอยู่ในสถานะว่าง (idle) ในช่วงเวลาที่ส่งการตอบกลับไปยังไคลเอนต์และรับคำขอถัดไปที่เซิร์ฟเวอร์

ปริมาณงานจะเพิ่มขึ้นเมื่อเราเพิ่มค่า **concurrency** เนื่องจาก Triton จะซ้อนทับการประมวลผลคำขอหนึ่งกับการสื่อสารของคำขออื่น

# แบบฝึกหัด

โปรดใช้เวลานี้เพื่อทดลองใช้เครื่องมือ **perf analyzer** คุณสามารถดูรายการพารามิเตอร์ทั้งหมดได้ด้วยอาร์กิวเมนต์ `--help` พารามิเตอร์บางส่วนที่เราแนะนำให้ลองใช้มีดังนี้:

* `-b <value>`: **ขนาดของแบตช์ (batch size)**
* `--concurrency-range <start:end:step>`: **ช่วงของค่าความพร้อมกัน (concurrency values) ที่จะทดสอบ**
* `--collect-metrics`: **เปิดใช้งานการเก็บรวบรวมเมตริก GPU**








## Model Analyzer (เครื่องมือวิเคราะห์โมเดล)

แม้จะอยู่นอกขอบเขตของหลักสูตรนี้ แต่เราอยากจะแนะนำเครื่องมือ **Model Analyzer** ซึ่งเป็นส่วนหนึ่งของ Triton เครื่องมือนี้จะทำการค้นหาการตั้งค่าพารามิเตอร์ต่างๆ เพื่อค้นหาพารามิเตอร์ที่เหมาะสมที่สุดที่ช่วยเพิ่มปริมาณงานการอนุมาน (inference throughput) ให้ได้สูงสุด ด้วยการประมวลผลเล็กน้อย ผลลัพธ์สามารถดูได้ในรูปแบบ PDF ด้วยเช่นกัน ไวยากรณ์สำหรับการรัน Model Analyzer แสดงอยู่ด้านล่าง พร้อมกับตัวอย่างของผลลัพธ์ Model Analyzer








In [ ]:
#%%bash
## Writing constraints to file
#cat > model_analyzer_constraints.yaml <<EOL 
#model_repository: /model_repository/
#triton_launch_mode: "local"
#latency_budget: 5
#run_config_search_max_concurrency: 64
#run_config_search_max_instance_count: 3
#run_config_search_max_preferred_batch_size: 8
#profile_models:
#  virus_prediction
#
#EOL

In [ ]:
# Run model_analyzer profiler on XGBoost model 
#!model-analyzer profile -f model_analyzer_constraints.yaml --override-output-model-repository

![image](images/model_analyzer.png)

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**ทำได้ดีมาก!** ไปยัง [สมุดบันทึกถัดไป](3-08_k-means_dask.ipynb) กันเลย